In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score

## Puntaje
Your submission scored 0.87686, which is an improvement of your previous score of 0.87516. Great job!

In [34]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')
#df19 eliminado, por el df21
df20 =  pd.read_csv('../data/featureUsuariosRegistraronActividadEl293031YUltimaHora.csv', low_memory=False, index_col='person')
df21 =  pd.read_csv('../data/feature_color_ult15dias.csv', low_memory=False, index_col='person')
df22 =  pd.read_csv('../data/featureUltimoEventoRealizadoPorElUsuario.csv', low_memory=False, index_col='person')
df23 =  pd.read_csv('../data/cant_max_eventos_sobre_un_mismo_telefono_ult_quincena.csv', low_memory=False, index_col='person')


labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
# Al que tenga "diferencia_de_count_de_eventos.csv" hay que rellenarlo
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17, how='inner')
df_unidos = df_unidos.join(df18, how='inner')
df_unidos = df_unidos.join(df20, how='inner')
df_unidos = df_unidos.join(df21, how='inner')
df_unidos = df_unidos.join(df22, how='inner')
df_unidos = df_unidos.join(df23, how='inner')

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape
#19414

(19414, 119)

In [35]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 90)

In [36]:
df_unidos

,tiene_checkouts,llegaron_por_ad,llegaron_por_search,tiene_conversions,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_visitas,cant_leads,cant_brand_listings,cant_generic_listings,cant_entradas_ads,cant_entradas_buscador,total_sesiones,promedio_eventos_por_sesion,checkout_Apple,checkout_Asus,checkout_LG,checkout_Lenovo,checkout_Motorola,checkout_Quantum,checkout_Samsung,checkout_Sony,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony,dispositivo_Computer,dispositivo_Smartphone,dispositivo_Tablet,dispositivo_Unknown,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,compro_onsite,cant_eventos_4_a_10hs,cant_eventos_11_a_14hs,cant_eventos_15_a_20hs,cant_eventos_21_a_3hs,storage_mas_visto,storage_mas_checkout,storage_mas_eventos,suma_total,EsDeSaoPablo,cantidadMaximaQueVeUnMismoProducto,ult_quin_cant_conversions,ult_quin_cant_checkouts,ult_quin_cant_viewed_product,ult_quin_cant_searched_product,ult_quin_cant_visitas,ult_quin_cant_leads,ult_quin_cant_brand_listings,ult_quin_cant_generic_listings,ult_quin_cant_entradas_ads,ult_quin_cant_entradas_buscador,tasa_ult_quin_cant_checkouts,tasa_ult_quin_cant_conversions,tasa_ult_quin_cant_viewed_product,tasa_ult_quin_cant_searched_product,tasa_ult_quin_cant_visitas,tasa_ult_quin_cant_leads,tasa_ult_quin_cant_brand_listings,tasa_ult_quin_cant_generic_listings,tasa_ult_quin_cant_entradas_ads,tasa_ult_quin_cant_entradas_buscador,ult_quin_total_sesiones,ult_quin_promedio_eventos_por_sesion,ult_checkout,ult_conversion,ult_viewed product,ult_searched products,ult_visited site,ult_lead,ult_brand listing,ult_generic listing,ult_ad campaign hit,ult_search engine hit,days_elapsed,realiza30EventosEn20MinsLosUltimos2Dias,model_encoding,diff_checkouts_15_dias,diff_visited_site_15_dias,diff_searched_products_15_dias,diff_viewed_products_15_dias,total_top_10_phones_checkout,different_top_10_phones_checkout,ult_quin_cant_eventos_4_a_10hs,ult_quin_cant_eventos_11_a_14hs,ult_quin_cant_eventos_15_a_20hs,ult_quin_cant_eventos_21_a_3hs,RealizoActividadEl30,RealizoActividadEl31,RealizoActividadEl31UltimaHora,RealizoActividadEl29,color_checkout,color_conversion,color_viewed product,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,cantidadMaximaDeTelefonoVisto
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,True,False,True,False,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1,9.000000,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,0,1,0,0,1,0,0,0,0,0,0,False,0.0,0.0,0.0,9.0,4.0,4.0,4.0,12.0,False,4.0,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,100.000000,0.0,400.000000,100.000000,100.000000,0.0,0.000000,100.000000,0.000000,100.000000,1.0,9.000000,20180518001156,0,20180518003030,20180518001052,20180518000722,0,0,20180518000722,0,20180518000723,13.994884,False,141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,False,False,False,False,7921.0,0.0,8937.0,0,0,0,0,0,0,0,0,0,1,0,0.0
ad93850f,True,True,True,False,0.0,1.0,20.0,0.0,5.0,0.0,15.0,7.0,10.0,7.0,5,13.000000,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0,1,0,0,1,0,0,0,0,0,0,False,0.0,0.0,0.0,65.0,5.0,0.0,5.0,10.0,True,17.0,0.0,1.0,20.0,0.0,5.0,0.0,15.0,7.0,10.0,7.0,100.000000,0.0,2000.000000,0.000000,500.000000,0.0,1500.000000,700.000000,1000.000000,700.000000,5.0,13.000000,20180514235419,0,20180518221245,0,20180522224131,0,20180518221254,20180522225054,20180522225051,20180522225051,17.003947,False,57,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,65.0,False,False,False,False,0.0,0.0,4015.0,0,0,0,0,1,0,0,0,0,0,0,0.0
0297fc1e,True,True,False,False,0.0,7.0,404.0,6.0,95.0,1.0,4.0,21.0,29.0,0.0,89,6.370787,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0,1,0,0,0,1,0,0,0,0,0,F

In [37]:
features = df_test.columns.tolist()
features.remove('label')

In [38]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [39]:
param_grid = { 
    'n_estimators': [x for x in range(100, 500, 100)],
    'gamma' : [1,2,3,4,5],
}

In [40]:
xgb = XGBClassifier(random_state=80)
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=4,)
CV_xgb.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [100, 200, 300, 400], 'gamma': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [41]:
CV_xgb.best_params_

{'gamma': 5, 'n_estimators': 100}

In [42]:
CV_xgb.best_score_
#0.8751560441726721

0.8735495828851462

In [43]:
model_xgb = XGBClassifier(scale_pos_weight=1, n_estimators=100, random_state=80, max_depth=3)
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict_proba(X_test)
y_pred_proba = [p[1] for p in y_pred]
print(roc_auc_score(y_test, y_pred_proba))
#0.8577688383045525

0.8552008110936682


In [45]:
#  Acá quedan ordenadas las features por importancia para el modelo, aparentemente la marca más vista y la marca
#  que más compró cada usuario no es tan importante.
feature_importances = pd.DataFrame(CV_xgb.best_estimator_.feature_importances_,\
                                   index = X.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(50)

,importance
cant_viewed_product,0.062977
color_checkout,0.055344
ult_conversion,0.051527
days_elapsed,0.047710
ult_viewed product,0.043893
tasa_ult_quin_cant_checkouts,0.041985
dispositivo_Computer,0.040076
ult_checkout,0.034351
cant_eventos_21_a_3hs,0.032443
tasa_ult_quin_cant_entradas_ads,0.026718


In [46]:
top_40_features = feature_importances.index.tolist()[:40]

In [47]:
X = df_test[top_40_features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [48]:
param_grid = { 
    'n_estimators': [x for x in range(100, 600, 100)],
    'gamma' : [1,2,3,4,5],
}

xgb_2 = XGBClassifier(random_state=80)
CV_xgb_2 = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=4,)
CV_xgb_2.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [100, 200, 300, 400, 500], 'gamma': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [49]:
CV_xgb_2.best_params_

{'gamma': 4, 'n_estimators': 100}

In [50]:
CV_xgb_2.best_score_
#0.8757367316054401
#0.8761497674857208

0.8751144586588252

In [51]:
print(classification_report(y_test, CV_xgb_2.predict(X_test)))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      3675
          1       0.50      0.03      0.05       208

avg / total       0.92      0.95      0.92      3883



C:\Users\jonat\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [52]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos[top_40_features], how='inner')

In [53]:
df_events.shape

(19415, 40)

In [54]:
# Notar que la predicción está hecha con el modelo pasado por GridSearchCV y no el otro que está sin kfoldear.
kaggle_pred = CV_xgb_2.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [55]:
df_submit.to_csv('../submit_v13.csv')